<a href="https://colab.research.google.com/github/varshachawan/SentimentClassifierTF/blob/master/sentimentAnalysis_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

In [0]:
from __future__ import absolute_import,division,print_function,unicode_literals


In [0]:
try :
  !pip uninstall tb-nightly tensorboardx tensorboard
  !pip install tf-nightly
except Exception:
  pass
  

In [0]:
import tensorflow as tf
import os
import datetime
import tensorflow_datasets as tfds

In [0]:
%load_ext tensorboard

In [0]:
import pkg_resources
for entry_point in pkg_resources.iter_entry_points('tensorboard_plugins'):
  print(entry_point.dist)

In [0]:
#!ls -alrt /usr/local/lib/python3.6/dist-packages/tensorboard*

In [0]:
! rm -r /usr/local/lib/python3.6/dist-packages/tensorboardcolab-0.0.22.dist-info


In [0]:
print(tf.__version__)

In [0]:
tf.config.experimental.list_physical_devices()

In [0]:
dataset,info = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00',with_info= True)

In [0]:
train_dataset = dataset['train']

In [0]:
info

In [0]:
len(list(train_dataset))

In [0]:
print(train_dataset)

In [0]:
BUFFER_SIZE = 30000
BATCH_SIZE = 128

In [0]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE,reshuffle_each_iteration=False)

In [0]:
for reviews in train_dataset.take(1):
  print(reviews)

In [0]:
for reviews in train_dataset.take(5):
  review_text = reviews['data']
  print(review_text.get('review_body').numpy())
  print(review_text.get('star_rating'))
  print(tf.where(review_text.get('star_rating')>3,1,0).numpy())

In [0]:
tokenizer = tfds.features.text.Tokenizer()
vocabulary_set= set()
for _,reviews in train_dataset.enumerate():
  review_text = reviews['data']
  reviews_tokens = tokenizer.tokenize(review_text.get('review_body').numpy())
  vocabulary_set.update(reviews_tokens)



In [0]:
vocab_size=len(vocabulary_set)
vocab_size

In [0]:
print(vocabulary_set)

In [0]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [0]:
for reviews in train_dataset.take(2):
    review_text = reviews['data']
    enc_exp= encoder.encode(review_text.get('review_body').numpy())
    print(review_text.get('review_body').numpy())
    print(enc_exp)

for index in enc_exp :
  print('{}----->{}'.format(index,encoder.decode([index])))

def encode(text_tensor,label_tensor):
  encoded_text = encoder.encode(text_tensor.numpy())
  label = tf.where(label_tensor>3,1,0)
  return encoded_text, label


In [0]:
def encode_map_fn(tensor):
  text = tensor['data'].get('review_body')
  label = tensor['data'].get('star_rating')
  encoded_text,label = tf.py_function(encode,inp=[text,label],Tout=(tf.int64,tf.int32))
  encoded_text.set_shape([None])
  label.set_shape([])
  return encoded_text , label

In [0]:
ar_encoded_data = train_dataset.map(encode_map_fn)


In [0]:
for fo,f1 in ar_encoded_data.take(2) :
  print('{}------->{}'.format(fo,f1))

In [0]:
TAKE_SIZE = 10000
train_data = ar_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)
test_data = ar_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)


In [0]:
vocab_size+=1

In [0]:
sample_text , sample_label = next(iter(test_data))
sample_text[0],sample_label[0]

In [0]:
for fo, f1 in test_data.take(2):
  print(tf.unique_with_counts(f1)[2].numpy())

In [0]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size,128))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences= True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences= True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [0]:
!rm -r /tmp/logs/

rm: cannot remove '/tmp/logs/': No such file or directory


In [0]:
logdir=os.path.join("/tmp/logs" ,datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback =tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='/temp/sentiment_analysis.hdf5',verbose=1,save_best_only=True)
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])
history = model.fit(train_data,epochs=4,validation_data=test_data,callbacks=[tensorboard_callback,checkpointer])
model.save('/tmp/final_sentiment_analysis.hdf5')


Epoch 1/4
    742/Unknown - 908s 1s/step - loss: 0.3592 - accuracy: 0.8339
Epoch 00001: val_loss improved from inf to 0.38129, saving model to /temp/sentiment_analysis.hdf5
742/742 [==============================] - 954s 1s/step - loss: 0.3592 - accuracy: 0.8339 - val_loss: 0.3813 - val_accuracy: 0.8587
Epoch 2/4
742/742 [==============================] - ETA: 0s - loss: 0.2545 - accuracy: 0.8934
Epoch 00002: val_loss improved from 0.38129 to 0.30234, saving model to /temp/sentiment_analysis.hdf5
742/742 [==============================] - 944s 1s/step - loss: 0.2545 - accuracy: 0.8934 - val_loss: 0.3023 - val_accuracy: 0.8716
Epoch 3/4
742/742 [==============================] - ETA: 0s - loss: 0.1979 - accuracy: 0.9213
Epoch 00003: val_loss improved from 0.30234 to 0.29481, saving model to /temp/sentiment_analysis.hdf5
742/742 [==============================] - 946s 1s/step - loss: 0.1979 - accuracy: 0.9213 - val_loss: 0.2948 - val_accuracy: 0.8823
Epoch 4/4
742/742 [==================

In [7]:
!ls -alrt /

total 7908
drwxr-xr-x   2 root root    4096 Apr 24  2018 home
drwxr-xr-x   2 root root    4096 Apr 24  2018 boot
drwxr-xr-x   2 root root    4096 Oct 29 21:25 srv
drwxr-xr-x   2 root root    4096 Oct 29 21:25 mnt
drwxr-xr-x   2 root root    4096 Oct 29 21:25 media
drwxr-xr-x   2 root root    4096 Oct 29 21:25 lib64
drwxr-xr-x   2 root root    4096 Mar  3 18:00 lib32
drwxr-xr-x   1 root root    4096 Mar  3 18:03 run
drwxr-xr-x   1 root root    4096 Mar  3 18:03 bin
drwxr-xr-x   1 root root    4096 Mar  3 18:04 sbin
drwxr-xr-x   1 root root    4096 Mar  3 18:05 lib
drwxr-xr-x   1 root root    4096 Mar  3 18:11 content
-rw-r--r--   1 root root 3997028 Mar  3 18:11 dlib-19.18.0-cp36-cp36m-linux_x86_64.whl
-rw-r--r--   1 root root 3989833 Mar  3 18:11 dlib-19.18.0-cp27-cp27mu-linux_x86_64.whl
drwxr-xr-x   4 root root    4096 Mar  3 18:38 tensorflow-2.1.0
drwxr-xr-x   4 root root    4096 Mar  3 18:43 swift
drwxr-xr-x   1 root root    4096 Mar  3 18:43 tools
drwxr-xr-x   1 root root    4096 M

In [0]:
eval_loss, eval_acc = model.evaluate(test_data)
print('/nEval_loss:{:.3f},Eval_acc:{:.3f}'.format(eval_loss,eval_acc) )

In [0]:
for fo,f1 in test_data.take(2):
  print(f1)
  print(model.predict(fo))
  

In [0]:
model.layers

In [0]:
model.summary()

In [0]:
model.get_layer('embedding').output


In [0]:
!nvdia-smi

In [0]:
import matplotlib as plt
def plot_graph(history,metric):
  plt.plot(history.hstory['meric'])


# New Section